In [7]:
from transformers import GPT2LMHeadModel, PreTrainedTokenizerFast

# Ko-GPT 모델과 토크나이저 로드
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
                                                    bos_token='</s>', eos_token='</s>', unk_token='<unk>',
                                                    pad_token='<pad>', mask_token='<mask>')

model = GPT2LMHeadModel.from_pretrained("skt/kogpt2-base-v2")

def generate_subheadings(intro_text, max_new_tokens=10, num_return_sequences=3):
    # 자기소개서 텍스트를 토크나이즈
    inputs = tokenizer.encode(intro_text, return_tensors='pt')

    # 모델을 사용해 텍스트 생성 (do_sample=True로 샘플링 활성화)
    outputs = model.generate(inputs, 
                             max_new_tokens=max_new_tokens, # 새로 생성할 토큰 수 제한
                             num_return_sequences=num_return_sequences, 
                             no_repeat_ngram_size=2, 
                             early_stopping=True,
                             do_sample=True,    # 샘플링 방식 활성화
                             top_k=50,          # 샘플링 시 고려할 상위 K개의 선택지
                             top_p=0.95         # 누적 확률 0.95를 넘는 선택지만 고려
                             )

    # 결과 디코딩
    generated_subheadings = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]

    return generated_subheadings

# 테스트할 자기소개서 텍스트
intro_text = """
저는 책임감과 끈기를 바탕으로 목표를 이루기 위해 노력하는 사람입니다. 
다양한 프로젝트 경험을 통해 문제 해결 능력을 키워왔으며, 이를 바탕으로 귀사의 경영 재무관리 부서에서 기여하고자 합니다.
"""

# 소제목 생성
subheadings = generate_subheadings(intro_text)
for i, subheading in enumerate(subheadings):
    print(f"Generated Subheading {i+1}: {subheading}")


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


Generated Subheading 1: 
저는 책임감과 끈기를 바탕으로 목표를 이루기 위해 노력하는 사람입니다. 
다양한 프로젝트 경험을 통해 문제 해결 능력을 키워왔으며, 이를 바탕으로 귀사의 경영 재무관리 부서에서 기여하고자 합니다.
특히 재무관리와 관련된 일들은 꾸준히 하고 싶고
Generated Subheading 2: 
저는 책임감과 끈기를 바탕으로 목표를 이루기 위해 노력하는 사람입니다. 
다양한 프로젝트 경험을 통해 문제 해결 능력을 키워왔으며, 이를 바탕으로 귀사의 경영 재무관리 부서에서 기여하고자 합니다.
귀사가 가장 중점적으로 하고 있는 사업은 바로 `
Generated Subheading 3: 
저는 책임감과 끈기를 바탕으로 목표를 이루기 위해 노력하는 사람입니다. 
다양한 프로젝트 경험을 통해 문제 해결 능력을 키워왔으며, 이를 바탕으로 귀사의 경영 재무관리 부서에서 기여하고자 합니다.
저는 경영자로서의 책임을 성실히 수행하는 사람입니다.
그


### 프롬프팅 엔지니어링

In [17]:
from transformers import GPT2LMHeadModel, PreTrainedTokenizerFast

# Ko-GPT 모델과 토크나이저 로드
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
                                                    bos_token='</s>', eos_token='</s>', unk_token='<unk>',
                                                    pad_token='<pad>', mask_token='<mask>')

model = GPT2LMHeadModel.from_pretrained("skt/kogpt2-base-v2")

def generate_subheadings(intro_text, max_new_tokens=15, num_return_sequences=3):
    prompt = "주어진 자기소개서를 요약해 제목만 읽어도 바로 이해할 수 있도록 필요한 내용만 넣어 자기소개서의 소제목을 생성해줘 \n"
    full_text = prompt + intro_text
    inputs = tokenizer.encode(full_text, return_tensors='pt')

    outputs = model.generate(inputs, 
                             max_new_tokens=max_new_tokens, # 새로 생성할 토큰 수 제한
                             num_return_sequences=num_return_sequences, 
                             no_repeat_ngram_size=2, 
                             early_stopping=True,
                             do_sample=True,    # 샘플링 방식 활성화
                             top_k=50,          # 샘플링 시 고려할 상위 K개의 선택지
                             top_p=0.95         # 누적 확률 0.95를 넘는 선택지만 고려
                             )

    generated_texts = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]

    new_subheadings = [text.replace(full_text, '').strip() for text in generated_texts]

    return new_subheadings

# 테스트할 자기소개서 텍스트
intro_text = """
흔히들 제가 믿음이 가는 사람이라고 많이 말합니다. 오래된 친구들은 어떠한 일에도 흔들림 없이 결정하는 모습이 익숙하지 않은 문제를
해결할 때 많은 의지가 된다고 평가합니다. 문제가 발생했을 때 새로운 해결 방안으로 확신을 가지고 문제를 해결하는 태도가 장점입니다.
이러한 성격은 팀으로 수행할 때 도움이 되었습니다. 프로젝트의 리더 역할을 맡았을 때 주제의 방향을 설정하고 이를 구체화하는 방법을
제안하여 팀원들에게 확신을 주었고 이로 인해 프로젝트는 안정적으로 진행되었고 졸업 설계가 무사히 진행될 수 있었습니다.
"""

# 소제목 생성
subheadings = generate_subheadings(intro_text)
for i, subheading in enumerate(subheadings):
    print(f"Generated Subheading {i+1}: {subheading}")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


Generated Subheading 1: 제가 경험한 사례입니다.
지난 1년은 힘들고 어려울 때마다 이 이야기를 꺼
Generated Subheading 2: 또한 이 프로젝트가 진행되면서 프로젝트 팀은 프로젝트를 진행하면서 느낀 어려움
Generated Subheading 3: 이런 팀원들의 노력으로 지금 대학원에 재학 중인 친구들이 자신의 진로에


### 퓨샷러닝

In [22]:
from transformers import GPT2LMHeadModel, PreTrainedTokenizerFast

# GPT 모델과 토크나이저 로드 (KoGPT2 사용)
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
                                                    bos_token='</s>', eos_token='</s>', unk_token='<unk>',
                                                    pad_token='<pad>', mask_token='<mask>')

model = GPT2LMHeadModel.from_pretrained("skt/kogpt2-base-v2")

def few_shot_subheading_generation(intro_text, examples, max_new_tokens=30):
    # 퓨샷러닝용 예시 제공
    prompt = "다음은 자기소개서의 문장과 그에 맞는 소제목 예시입니다:\n\n"
    
    # 주어진 예시를 프롬프트로 추가
    for example in examples:
        prompt += f"문장: {example['intro']}\n소제목: {example['subheading']}\n\n"
    
    # 새로운 입력 추가
    prompt += f"문장: {intro_text}\n소제목:"

    # 입력을 토크나이즈
    inputs = tokenizer.encode(prompt, return_tensors='pt')

    # 모델을 사용해 텍스트 생성
    outputs = model.generate(inputs, 
                             max_new_tokens=max_new_tokens, 
                             no_repeat_ngram_size=2, 
                             early_stopping=True,
                             do_sample=True,
                             top_k=50,
                             top_p=0.8)

    # 결과 디코딩
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # 생성된 텍스트에서 프롬프트 이후의 부분만 추출 (새로운 소제목)
    generated_subheading = generated_text.replace(prompt, '').strip()

    return generated_subheading

# 퓨샷러닝 예시 데이터
examples = [
    {"intro": """
    금융결제원은 국내 다양한 금융 관련 서비스를 지원하는 최고의 기업입니다.
    이런 여러 서비스 운영에 있어 중요한 것은 사용자에게 적절한 UI의 표출, 그리고 효율적인 데이터 처리를 통한 빠른 속도 제공입니다. 
    저는 이를 배우기 위해 매일 진행 중인 알고리즘 풀이 외에도 기존에 공부했던 JSP에 대해 조금 더 파악하고, spring에 활용하기 전에
    배우기 위해 관련 학원에 다니며 공부하였고, 여러 예제를 사용한 Spring 프로젝트를 진행했습니다. 
    그리고 JPA를 사용하는 여러 방식과 주의점, querydsl 등 많은 부분에의 공부를 해 보았습니다.
    백엔드 이외에도 front-end에 활용되는 javascript의 기능 활용을 더 잘 알기 위해, vanilla JS를 사용하여 조금씩 난도를 높여가며
    프로젝트를 진행 중입니다. 그렇게 실력을 쌓아 가던 중 운이 좋게도 그동안 제가 공부하였고 앞으로의 진로로 결정하게 된 java spring이
    기반이 되는 전산직 사무원을 채용한다는 소식을 들었고, 최고의 기업에서 가장 좋아하는 공부를 할 수 있을 것이라는 기대감을 품고
    지원하게 되었습니다.
    """,
     "subheading": "최고가 되기 위해서"},
    {"intro": """
    개발에 관한 학습과 이에 관한 프로젝트를 시도했고 이를 통해 Spring boot에 관한 기본적인 지식을 쌓을 수 있었습니다. 
    제가 가장 좋아하는 Spring boot 관련 개발 공부 외에도, 알고리즘 풀이를 위해 매일 java를 통한 문제 풀이를 진행하고 있으며,
    백엔드에 있어 필수적인 요소 중 하나인 DB에 관한 문제 풀이와 기초 공부를 해 왔습니다.
    또 javascript를 통한 몇 가지 프로젝트를 시도해 보며, 개발자로서 가진 지식의 넓이를 키우기 위한 사전 준비를 해왔습니다.
    """,
     "subheading": "배움과 기초"},
    {"intro": """
    저는 전자공학도로서 학교에서 여러 언어들을 배웠습니다.ATMega와 같은 어셈블리 언어, C언어나 C++ 언어를 처음에 배워 정확하고
    효율적인 코딩 방법을 알게 되었습니다.또, 아두이노나 라즈베리파이를 통한 실습을 진행하며 서버에서 어떤 식으로 데이터를 통신하는 것이
    좋은지 배웠습니다. 학교 외에서도 제 꿈을 위해 더 많은 것을 알기 위해 노력했습니다. 여러 사람들과 혹은 혼자 프로젝트를
    진행하며 항상 새로운 언어를 사용하며 많은 언어의 장단점과 사용 목적을 알았습니다. 한국에서 java가 발전되기 전에 사용되던
    PHP나 가장 기초적인 언어인 ASP를 다루어 보았습니다.MySql을 사용하여 프로그래머스 사이트의 모든 문제를 해결하였고,
    MS-Sql의 프로시저와 쿼리 모두를 활용해 보았습니다.
    그리고 가장 많은 노력을 기울인 것은 java 언어입니다.
    레거시 JSP를 이용한 페이지 제작을 해 보았고, 이 경험을 통해 spring boot를 사용하는 웹 페이지 제작을 진행 중이며,
    현재도 매일 알고리즘 풀이를 진행하고 있습니다.
    이렇게 저는 개발자로서 넓으며, 동시에 깊은 지식을 갖기 위한 노력을 항상 하고 있습니다.
    """,
     "subheading": "넓은, 동시에 깊은 지식"}
]

# 새로운 자기소개서 문장
intro_text = """
흔히들 제가 믿음이 가는 사람이라고 많이 말합니다. 오래된 친구들은 어떠한 일에도 흔들림 없이 결정하는 모습이 익숙하지 않은 문제를
해결할 때 많은 의지가 된다고 평가합니다. 문제가 발생했을 때 새로운 해결 방안으로 확신을 가지고 문제를 해결하는 태도가 장점입니다.
이러한 성격은 팀으로 수행할 때 도움이 되었습니다. 프로젝트의 리더 역할을 맡았을 때 주제의 방향을 설정하고 이를 구체화하는 방법을
제안하여 팀원들에게 확신을 주었고 이로 인해 프로젝트는 안정적으로 진행되었고 졸업 설계가 무사히 진행될 수 있었습니다.
"""

# 소제목 생성
generated_subheading = few_shot_subheading_generation(intro_text, examples)
print(f"Generated Subheading: {generated_subheading}")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


Generated Subheading: 새로운 지식에 대한 이해와
비전을 공유할 수 있는 방법들을 모색하고 있었으니 앞으로도 열심히 하겠습니다.
JSP :.
Cop


#### 결론
- 퓨샷 프롬프팅을 진행하였을시에 원하는 답변과 비슷한 답변이 아닌 아예 사용할 수 없을 정도의 답변이 나옴을 확인할 수 있음
- 기존의 퓨샷 프롬프팅보다는 일반적인 프롬프팅이 더 나아보임

### 파인튜닝
- 아직 데이터 셋을 확보하지 못해 실행하지 못함

In [ ]:
# from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments, TextDataset, DataCollatorForLanguageModeling

# # 1. 모델과 토크나이저 로드
# model = GPT2LMHeadModel.from_pretrained("skt/kogpt2-base-v2")
# tokenizer = GPT2Tokenizer.from_pretrained("skt/kogpt2-base-v2")

# # 2. 데이터셋 로드
# def load_dataset(file_path, tokenizer, block_size=128):
#     dataset = TextDataset(
#         tokenizer=tokenizer,
#         file_path=file_path,
#         block_size=block_size
#     )
#     return dataset

# train_dataset = load_dataset("train.txt", tokenizer)

# # 3. 데이터 보강 (자동 padding, masking 등)
# data_collator = DataCollatorForLanguageModeling(
#     tokenizer=tokenizer,
#     mlm=False,  # GPT는 Masked LM이 아님
# )

# # 4. 트레이닝 인자 설정
# training_args = TrainingArguments(
#     output_dir="./results",
#     overwrite_output_dir=True,
#     num_train_epochs=3,  # 학습 횟수
#     per_device_train_batch_size=4,
#     save_steps=500,
#     save_total_limit=2,
# )

# # 5. 트레이너 정의
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_dataset,
#     data_collator=data_collator,
# )

# # 6. 파인튜닝 실행
# trainer.train()

# # 7. 모델 저장
# model.save_pretrained("./fine_tuned_model")
# tokenizer.save_pretrained("./fine_tuned_model")


In [ ]:
# from transformers import GPT2LMHeadModel, GPT2Tokenizer

# # 파인튜닝된 모델과 토크나이저 로드
# model = GPT2LMHeadModel.from_pretrained("./fine_tuned_model")
# tokenizer = GPT2Tokenizer.from_pretrained("./fine_tuned_model")

# def generate_subheadings(intro_text, max_new_tokens=30, num_return_sequences=3):
#     prompt = "Input: " + intro_text + "\nOutput:"
    
#     inputs = tokenizer.encode(prompt, return_tensors='pt')
    
#     outputs = model.generate(
#         inputs, 
#         max_new_tokens=max_new_tokens,
#         num_return_sequences=num_return_sequences,
#         do_sample=True,
#         top_k=50,
#         top_p=0.95,
#     )

#     generated_texts = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]
#     subheadings = [text.replace(prompt, '').strip() for text in generated_texts]
    
#     return subheadings

# # 테스트할 자기소개서
# intro_text = """
# 저는 끈기와 성실함을 바탕으로 목표를 이루기 위해 노력해왔습니다. 다양한 프로젝트를 수행하며 문제 해결 능력을 배양하였습니다.
# """

# # 소제목 생성
# subheadings = generate_subheadings(intro_text)
# for i, subheading in enumerate(subheadings):
#     print(f"Generated Subheading {i+1}: {subheading}")

### 키워드와 자소서 투입시

In [33]:
from transformers import GPT2LMHeadModel, PreTrainedTokenizerFast

# Ko-GPT 모델과 토크나이저 로드
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
                                                    bos_token='</s>', eos_token='</s>', unk_token='<unk>',
                                                    pad_token='<pad>', mask_token='<mask>')

model = GPT2LMHeadModel.from_pretrained("skt/kogpt2-base-v2")

def generate_subheadings(intro_text, keywords, max_new_tokens=7, num_return_sequences=2):
    # 프롬프트에 키워드를 추가하여 GPT-2에게 힌트를 줌
    prompt = f"""
    주어진 자기소개서를 요약해, '{', '.join(keywords)}' 키워드를 바탕으로 소제목 생성해줘.
    
    예시는 투입될 자기소개서의 내용이 '제가 개발자가 되기로 결심한 뒤, 마음속에 항상 품고 있는 저의 신조입니다.남들보다 늦은 대학교 3학년 때에 
    저는 제 진로를 결심할 수 있었습니다. 그때에 저는 웹과 앱 만들기에 관심이 있었고, 두 가지 모두에 쓰이는 java 언어를 공부하기로
    마음먹었습니다. 이후로 프로젝트로 JSP로 웹을 만들고, kotlin을 통해 앱을 만들어 패션 추천 애플리케이션을 만들게 되었습니다.
    이 과정에서, backend 개발자가 되려는 결정을 하였고, 이후로는 그와 관련된 수많은 공부를 하였습니다.
    저보다 먼저 공부하고, 많은 것을 배워 둔 사람들과의 차이를 좁히기 위해 기초적인 언어를 알기 위해 java 알고리즘 풀이를
    매일 진행하고 있으며, 여러 백엔드 실습을 하였습니다.
    싸이버로지텍에서 제가 가진 역량을 통해 기여하고 함께 나아갈 수 있는 사람이 되겠습니다.'이고
    키워드는 '노력'일때
    
    출력되어야 할 소제목 즉 결과물은 '끊임 없이 노력하며 나아가는 삶'이다.
    """
    full_text = prompt + intro_text

    inputs = tokenizer.encode(full_text, return_tensors='pt')

    outputs = model.generate(inputs, 
                             max_new_tokens=max_new_tokens, # 새로 생성할 토큰 수 제한
                             num_return_sequences=num_return_sequences, 
                             no_repeat_ngram_size=1, 
                             early_stopping=True,
                             do_sample=True,    # 샘플링 방식 활성화
                             top_k=30,          # 샘플링 시 고려할 상위 K개의 선택지
                             top_p=0.95         # 누적 확률 0.95를 넘는 선택지만 고려
                             )

    generated_texts = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]

    new_subheadings = [text.replace(full_text, '').strip() for text in generated_texts]

    return new_subheadings

# 테스트할 자기소개서 텍스트
intro_text = """
흔히들 제가 믿음이 가는 사람이라고 많이 말합니다. 오래된 친구들은 어떠한 일에도 흔들림 없이 결정하는 모습이 익숙하지 않은 문제를
해결할 때 많은 의지가 된다고 평가합니다. 문제가 발생했을 때 새로운 해결 방안으로 확신을 가지고 문제를 해결하는 태도가 장점입니다.
이러한 성격은 팀으로 수행할 때 도움이 되었습니다. 프로젝트의 리더 역할을 맡았을 때 주제의 방향을 설정하고 이를 구체화하는 방법을
제안하여 팀원들에게 확신을 주었고 이로 인해 프로젝트는 안정적으로 진행되었고 졸업 설계가 무사히 진행될 수 있었습니다.
"""

# 소제목을 생성할 키워드
keywords = ["문제 해결", "리더십"]

# 소제목 생성
subheadings = generate_subheadings(intro_text, keywords)
for i, subheading in enumerate(subheadings):
    print(f"Generated Subheading {i+1}: {subheading}")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


Generated Subheading 1: 끝으로, 노하우를 갖춘 파트
Generated Subheading 2: 그리고 이렇게 좋은 결과가 나기까지 과정은


#### 결론
- 어느정도 자소서와 관련된 말은 생성이 되나 소제목에 적합하지는 X
- 파인 튜닝이 필요함

### 생각해 보아야 할 것
1. 파인 튜닝을 할 시 데이터셋은?
    - 파인튜닝을 진행할 시 원하는 성능이 나올 것인가? (데이터 셋 확보에 따라 다를 것으로 보임)
2. 투입을 자소서만 or 키워드+자소서